# BigBird Text classification Tutorial *Longformer*

*Big Bird*  is part of a new generation of Transformer based architectures that try to resolve the main limitation of attention mechanisms; the quadratic nature attention. These models try to solve this problem by fingding ways to approximate the full attention matrix In a previous [post](https://jesusleal.io/2020/10/20/https://jesusleal.io/2020/11/24/Longformer-with-IMDB/) I explored how to use Hugging Face Transformers library to fine tune a Longformer model in a simple multiclass classification setting. In this post I explore Big Bird available through the Hugging Face Transformers library. 

*Big Bird* expands the number of tokens that the model can process from 512 to 4096 tokens thanks to a new Attention mechanism known as *Block Sparse Attention*. This flavor of attention uses a combination of global attention (on selected tokends), window attention (just like LongFormer) and random attention. 



In traditional Transformer architectures each token attends to everyother other token and as result the complexity is n2. BigBird uses global attention on selected tokens such as `CLS` so that they can attend to all the tokens in the sequence, window attention where each token attends to $\frac{1}{2}\ w$ tokens to the left and to the right. tokens and random attention where each token follows random tokens.

The main idea behind this model is to use the notion of random walks in a small world network. 


libra  *Trainer* class to easily create a text classification pipeline. The code was pretty straighforward to implement and I was able to obtain results that put the basic model at a very competitive level with a few lines of code. In that post I also briefly discussed one of the main drawbacks of the first generation of Transformers and BERT based architectures; the sequence lenght is limited to a maximum of 512 characters. The reason behind that limitation is the fact that self-attention mechanism scale quadratically with the input sequence length *O(n^2)*. Given the need to process longer sequences of text a a second generation of attention based models have been proposed. The idea behind this models is to reduce the memory footprint of the attention mechanisms in order to process longer sequences of text; see this really useful analysis of transformer models that try to overcome this limitation [by researchers from Google](https://arxiv.org/pdf/2009.06732.pdf). New models such as the [***Reformer***](https://arxiv.org/pdf/2001.04451.pdf) by Google proposes a series of innovations to the traditional Transformer architecture such as Local Self Attention, Locality sensitive hashing (LSH) Self-Attention, Chunked Feed Forward Layers, etc. This [post](https://huggingface.co/blog/reformer) from Hugging Face for a detailed discussion). This model can process sequences of half a million tokens with as little as 8GB of RAM. However one big drawback of the model for downstream applications is the fact that the authors have not released pre trained weights of their model and at the time of publication of this post there is no freely available model pretrained on a large corpus. 

Another very promising model, and the subject of this post, is the [***Longformer***](https://arxiv.org/pdf/2004.05150.pdf) by researchers from Allen AI Institure. The Longformer allows the processing sequences of thousand characters without facing the memory bottleneck of BERT like architectures and achieved SOTA at the time of publication in several benchmarks. The authors use a new variation of attention, called local attention where every token only attends to tokens in the vicinity defined by a window *w* where each token attends to $\frac{1}{2}\ w$  tokens to the left and to the right. To increase the receptive field the authors also applied dilation to the local window so they can increase the size of w without incurring in memory costs. A dilation is simply a "hole", meaning the token simply skips that token thus allowing tokens to reach farther tokens. The performance is not hurt since the transformer architecture has multiple attention heads across multiple layers and the different layers and head learn and attend different properties of texts and tokens. In addition to the local attention the authors also included a token that is attended globally so it can be used in downstream taks, just like thee *CLS* token of BERT. One of the interesting aspects of this model is the fact that the authors created their own CUDA kernel to calculate the attention scores of the sliding window attention. This type of attention is more efficient since there are many zeros in the matrix, this operation is called  matrix banded multiplication, but is not implemented in Pytorch/Tensorflow. Thanks to our friends from Hugging Face an implementation with standard CUDA kernels is available altough it does not have all the capabilities the authors of the Longformer model describe in their paper it is suitable for finetuning [downtream tasks](https://github.com/allenai/longformer). 


The authors tested the model with an autoregressive model to process sequences of thousands of tokens, achieving state of the art on *text8* and *enwik8*. They also tested the model on downstream tasks by finetuning the model with the weights of RoBERTA to conduct masked token prediction (MLM) of one third of the real news dataset, and a third of the stories dataset.  The authors pretrained two variations of the model a base model (with 12 layers) and a large model (30 layers). Both models were trained for 65K gradient updates with sequences of length 4,096 and batch size 64. Once the pretraining was completed they tested the models on downstream tasks such as question answering, coreference resolution and document classification. The model achieved SOTA results on the WikiHop TriviaQA datasets and in the hyper partisan data. For the IMDB dataset the authors achieved 95.7 percent accuracy, a small increase from the 95.3 percent accuracy reported by RoBERTa. 

Given all this nice features I decided to try the model and see how it compares to RoBERTA on the IMDB the iris dataset of text classification. For this script I used the trainer class from Hugging Face and the pretrained model offered by Allen AI available in the model hub of Hugging Face.


In [1]:
import torch
import datasets
import transformers
import pandas as pd
import numpy as np
from torch.nn import BCEWithLogitsLoss
from transformers import BigBirdTokenizer, \
BigBirdForSequenceClassification, Trainer, TrainingArguments,EvalPrediction, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import wandb
import random

In [2]:
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'], 
                                             cache_dir='/media/data_files/github/website_tutorials/data')

Reusing dataset imdb (/media/data_files/github/website_tutorials/data/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)


In [3]:
# load model and tokenizer and define length of the text sequence
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base',
                gradient_checkpointing=False,
                num_labels = 2,
                cache_dir='/media/data_files/github/website_tutorials/data',
                return_dict=True)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassifica

In [4]:
model

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0): BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [5]:
tokenizer = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', 
                                          max_length = 1024,
                                          cache_dir='/media/data_files/github/website_tutorials/data',)

One of the cool features about this model is that you can specify the attention sliding window across different levels; the authors exploited this design for the autoregressive language model using different sliding windows for different layers. If this parameter is not changed it will assume a default of 512 across all the different layers.

wandb.login()

In [6]:
train_data
#dir(train_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

For my implementation of the model, and to save speed in the pretraining I chose the maximun length of 1024 characters which covers close to 98 percent of all the documents in the dataset. Before using my brand new and still pretty much impossible to find RTX3090, I set the gradient checkpointing parameter to true. This saves a huge amount of memory and allows models such as the longformer to train on more modest GPU's such as my old EVGA GeForce GTX 1080. Gradient checkpointing is a really nice way to re use weights in the neural network and allows massive models to run on more modest settings with a 30 percent increase in training time. The original paper discussing gradient checkpointing can be found [here](https://arxiv.org/pdf/1604.06174.pdf) and a nice [discussion of gradient checkpointing can be hound here](https://qywu.github.io/2019/05/22/explore-gradient-checkpointing.html). 

Additionally to save memory and increase training time I also used mixed precision training to speed up the computation time of the training process. If you want to learn more about mixed precision I recommend this [blogpost](https://jonathan-hui.medium.com/mixed-precision-in-deep-learning-67f6dce3e0f3). With the combination of mixed precision, gradient accumulation and gradient checkpoint you can set the length to 4096. 

In [7]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 1024)

train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

In [8]:
# we make sure our truncation strateging and the padding are set to the maximung length
len(train_data['input_ids'][0])

1024

In [9]:
#train_data['input_ids'][1]

Once the tokenization process is finished we can use the set the column names and types. One thing that is important to note is that the `LongformerForSequenceClassification` implementation by default sets the global attention to the `CLS`[token](https://huggingface.co/transformers/_modules/transformers/modeling_longformer.html#LongformerForSequenceClassification), so there is no need to further modify the inputs.

In [10]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [11]:
# define accuracy metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # argmax(pred.predictions, axis=1)
    #pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In the paper the authors trained for 15 epochs, with batch size of 32, learning rate of 3e-5 and linear warmup steps equal to 0.1 of the total training steps. For this quick tutorial I went for the default learning rate of the trainer class which is 5e-5, 5 epochs for training, batch size of 8 with gradient accumulation of 8 steps for an effective batch size of 64 and 200 warm up steps (roughly 10 percent of total training steps). The overall training time for this implementation was 2 hours and 54 minutes.

In [12]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = '/media/data_files/github/website_tutorials/results',
    num_train_epochs = 4,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 32,    
    per_device_eval_batch_size= 16,
    evaluation_strategy = "epoch",
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=160,
    weight_decay=0.01,
    logging_steps = 4,
    learning_rate = 1e-5,
    fp16 = True,
    logging_dir='/media/data_files/github/website_tutorials/logs',
    dataloader_num_workers = 0,
    run_name = 'bigbird_classification_1e5'
)

In [13]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
# train the model
trainer.train()

wandb: Currently logged in as: jlealtru (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
0,0.200700,0.144661,0.948120,0.948304,0.944952,0.951680,541.431500,46.174000
1,0.085400,0.126741,0.955120,0.955012,0.957315,0.952720,542.147400,46.113000
2,0.077500,0.136603,0.954800,0.955098,0.948839,0.961440,562.767600,44.423000
3,0.101100,0.148595,0.955120,0.955352,0.950435,0.960320,595.682300,41.969000


TrainOutput(global_step=1560, training_loss=0.15553927932364436, metrics={'train_runtime': 20554.9647, 'train_samples_per_second': 0.076, 'total_flos': 7.86491631378432e+16, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 1244717056, 'init_mem_gpu_alloc_delta': 512766464, 'init_mem_cpu_peaked_delta': 392011776, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1416474624, 'train_mem_gpu_alloc_delta': 2122296320, 'train_mem_cpu_peaked_delta': 309325824, 'train_mem_gpu_peaked_delta': 3428258816})

After the training has been completed we can evaluate the performance of the model and make sure we are loading the right model.

In [15]:
# save the best model
trainer.model.save_pretrained('/media/data_files/github/website_tutorials/results/bigbird_base_imdb_1048_1e5')
tokenizer.save_pretrained('/media/data_files/github/website_tutorials/results/bigbird_base_imdb_1048_1e5')

('/media/data_files/github/website_tutorials/results/bigbird_base_imdb_1048_1e5/tokenizer_config.json',
 '/media/data_files/github/website_tutorials/results/bigbird_base_imdb_1048_1e5/special_tokens_map.json',
 '/media/data_files/github/website_tutorials/results/bigbird_base_imdb_1048_1e5/spiece.model',
 '/media/data_files/github/website_tutorials/results/bigbird_base_imdb_1048_1e5/added_tokens.json')

In [16]:
trainer.evaluate()

{'eval_loss': 0.1267414391040802,
 'eval_accuracy': 0.95512,
 'eval_f1': 0.9550120288692863,
 'eval_precision': 0.957315112540193,
 'eval_recall': 0.95272,
 'eval_runtime': 594.4082,
 'eval_samples_per_second': 42.059,
 'epoch': 4.0,
 'eval_mem_cpu_alloc_delta': -259088384,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 259088384,
 'eval_mem_gpu_peaked_delta': 2471463424}


The best iteration of our model achieved an accuracy 0.9534, below what the authors report (0.957). This results are probably explained by the fact that we have used several tricks to increase training speed, the use of half-precision floating-point (*fp16*) and the fact that we are not using their special CUDA kernel. Additionally 
As the authors recognize in the paper this corpus collection is composed mostly of shorter documents thus the model does not fully utilizes its capabilities to learn long sequences. Recent evaluations of the new generation of the model indicate that while longformer does not achieve the best results in any category it performs competitively across all the different tasks explored in the model evaluation, ranking second overall.

![Results](images/longformer_eval_accuracy_imdb.svg)

Thats it for this tutorial, hopefully you will find this helpful.